
Пропуски 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
matplotlib.style.use('ggplot')
%matplotlib inline
!pip install scikit-learn==0.22.1
!pip install impyute
import impyute as impy
!pip install missingpy
from missingpy import MissForest

In [ ]:
df = pd.read_csv('bank-full.csv', sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


#Подготавливаем данные

Необходимо заменить текстовые поля числами. Сделаю это с помощью "ключей"

In [ ]:
jobs = list(np.unique(df['job']))
fam = list(np.unique(df['marital']))
ed =list(np.unique(df['education']))
dflt =list(np.unique(df['default']))
hous = list(np.unique(df['housing']))
loan = list(np.unique(df['loan']))
contact = list(np.unique(df['contact']))
month = list(np.unique(df['month']))
poutcome = list(np.unique(df['poutcome']))
y = ['no', 'yes']
df1 = df

In [ ]:
for i in range (45211):
   df1['job'][i] = float(jobs.index(df['job'][i]))
   df1['marital'][i] = float(fam.index(df['marital'][i]))
   df1['education'][i] = float(ed.index(df['education'][i]))
   df1['housing'][i] = float(hous.index(df['housing'][i]))
   df1['loan'][i] = float(loan.index(df['loan'][i]))
   df1['contact'][i] = float(contact.index(df['contact'][i]))
   df1['month'][i] = float(month.index(df['month'][i]))
   df1['poutcome'][i] = float(poutcome.index(df['poutcome'][i]))
   df1['default'][i] = float(dflt.index(df['default'][i]))
   df1['y'][i] = float(y.index(df['y'][i]))
df1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17,9,977,3,-1,0,3,1
45207,71,5,0,0,0,1729,0,0,0,17,9,456,2,-1,0,3,1
45208,72,5,1,1,0,5715,0,0,0,17,9,1127,5,184,3,2,1
45209,57,1,1,1,0,668,0,0,1,17,9,508,4,-1,0,3,0


Выделяем часть датасета для работы

In [ ]:
X = df1.iloc[:, :-1].values  

Заменяем значения 

In [ ]:
def make_missing_value(X, del_fraction):
    N = X.shape[0]
    D = X.shape[1]
    delete_mask = np.array(np.random.random((N, D)))
    #Заполняем обозначенную долю True - будет nan, остальное False 
    #while (sum(sum(delete_mask))!= del_fraction*N*D): первый вариант не вариант, слишком долго
     # delete_mask = np.array(np.random.random((N, D)) < del_fraction)
    mas = []
    for i in range(45211):
      if i < int(45211*0.1):
        mas.append(bool(True))
      else:
        mas.append(bool(False))
    delete_mask = pd.DataFrame()
    for i in range (16):
      np.random.shuffle(mas)
      delete_mask[i] = mas

    # Значения где был True заменяются на nan, остальное остается значениями
    new_X = np.array(X)
    new_X = new_X.astype('float64')
    new_X[delete_mask] = np.nan
    return new_X

"Удаляем" долю результатов и при помощи impyute заменяем их на предполагаемые значения. Считаем точность, СКО, дисперсию. Применяем данную схему для заполнения средним и knn для всех вариантов количества пропусков.

In [ ]:
tt = X
tt = make_missing_value(tt, 0.1)
means = impy.mean(tt)
y = (np.isclose(X.astype('float64'), means))
acc_10_means = (45211*16*0.1-(45211*16 - np.sum(y)))/(45211*16*0.1)
disp_10_means = np.var(means)
sko_10_means = np.std(means)

In [ ]:
tt = X
tt = make_missing_value(tt, 0.1)
knn = impy.fast_knn(tt)
y = (np.isclose(X.astype('float64'), knn))
acc_10_knn = (45211*16*0.1-(45211*16 - np.sum(y)))/(45211*16*0.1)
disp_10_knn = np.var(knn)
sko_10_knn = np.std(knn)

Замена случайным лесом. Документация https://pypi.org/project/missingpy/

Для разнообразия методы заполнения взяты из разных библиотек

In [ ]:
tt = X
tt = make_missing_value(tt, 0.1)
imputer = MissForest()
rf = imputer.fit_transform(tt)
y = (np.isclose(X.astype('float64'), rf))
acc_10_rf = (45211*16*0.1-(45211*16 - np.sum(y)))/(45211*16*0.1)
disp_10_rf = np.var(rf)
sko_10_rf = np.std(rf)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


Повторяем для 25%

In [ ]:
tt = X
tt = make_missing_value(tt, 0.25)
means = impy.mean(tt)
y = (np.isclose(X.astype('float64'), means))
acc_25_means = (45211*16*0.25-(45211*16 - np.sum(y)))/(45211*16*0.25)
disp_25_means = np.var(means)
sko_25_means = np.std(means)

In [ ]:
tt = X
tt = make_missing_value(tt, 0.25)
knn = impy.fast_knn(tt)
y = (np.isclose(X.astype('float64'), knn))
acc_25_knn = (45211*16*0.25-(45211*16 - np.sum(y)))/(45211*16*0.25)
disp_25_knn = np.var(knn)
sko_25_knn = np.std(knn)

In [ ]:
tt = X
tt = make_missing_value(tt, 0.25)
rf = imputer.fit_transform(tt)
y = (np.isclose(X.astype('float64'), rf))
acc_25_rf = (45211*16*0.25-(45211*16 - np.sum(y)))/(45211*16*0.25)
disp_25_rf = np.var(rf)
sko_25_rf = np.std(rf)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6


Повторяем для 30%

In [ ]:
tt = X
tt = make_missing_value(tt, 0.3)
means = impy.mean(tt)
y = (np.isclose(X.astype('float64'), means))
acc_30_means = (45211*16*0.3-(45211*16 - np.sum(y)))/(45211*16*0.3)
disp_30_means = np.var(means)
sko_30_means = np.std(means)

In [ ]:
tt = X
tt = make_missing_value(tt, 0.3)
knn = impy.fast_knn(tt)
y = (np.isclose(X.astype('float64'), knn))
acc_30_knn = (45211*16*0.3-(45211*16 - np.sum(y)))/(45211*16*0.3)
disp_30_knn = np.var(knn)
sko_30_knn = np.std(knn)

In [ ]:
tt = X
tt = make_missing_value(tt, 0.3)
rf = imputer.fit_transform(tt)
y = (np.isclose(X.astype('float64'), rf))
acc_30_rf = (45211*16*0.3-(45211*16 - np.sum(y)))/(45211*16*0.3)
disp_30_rf = np.var(rf)
sko_30_rf = np.std(rf)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7


Сводня таблица результатов 

In [ ]:
table = pd.DataFrame(index = ['means, 10%', 'knn, 10%','rf, 10%', 'means, 25%', 'knn, 25%','rf, 25%',
                              'means, 30%', 'knn, 30%', 'rf, 30%'])

table['acc'] = [acc_10_means, acc_10_knn, acc_10_rf,
                acc_25_means, acc_25_knn, acc_25_rf,
                acc_30_means, acc_30_knn, acc_30_rf]
table['disp'] = [disp_10_means, disp_10_knn, disp_10_rf,
                 disp_25_means, disp_25_knn, disp_25_rf,
                 disp_30_means, disp_30_knn, disp_30_rf]
table['sko'] = [sko_10_means, sko_10_knn, sko_10_rf,
                sko_25_means, sko_25_knn,sko_25_rf,
                sko_30_means, sko_30_knn, sko_30_rf]
print('sko X ', np.std(X),'  disp X', np.var(X))
table

sko X  832.3328376979256   disp X 692777.9527102814


,acc,disp,sko
"means, 10%",0.000022,639496.286183,799.685117
"knn, 10%",0.179099,635915.955818,797.443387
"rf, 10%",0.214917,652140.925965,807.552429
"means, 25%",0.600009,638286.185332,798.928148
"knn, 25%",0.671197,615871.787668,784.774992
"rf, 25%",0.686890,637321.040681,798.323895
"means, 30%",0.666674,630399.041230,793.976726
"knn, 30%",0.726279,633171.938244,795.721018
"rf, 30%",0.738485,642504.165868,801.563576
